In [1]:
import pandas as pd
import spacy
import en_core_web_sm
import random
from spacy import displacy
import benepar

# parse
from collections import deque

In [2]:
df = pd.read_json('../data/reviewSelected100.json', lines=True, encoding = "ISO-8859-1")

In [3]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11


In [ ]:
# using spacy

In [1]:
!pip install benepar

In [12]:
benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/cecilialee/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


True

In [6]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

In [8]:
# try shallow prasing / chunking
conference_text = ("US unveils world's most powerful supercomputer, beats china")
conference_doc = nlp(conference_text)

In [9]:
sent = list(conference_doc.sents)[0]
print(sent._.parse_string)

(S (NP (NNP US)) (VP (VP (VBZ unveils) (NP (NP (NN world) (POS 's)) (ADJP (RBS most) (JJ powerful)) (NN supercomputer))) (, ,) (VP (VBZ beats) (NP (NN china)))))


In [10]:
print(list(conference_doc.sents)[0])

US unveils world's most powerful supercomputer, beats china


In [11]:
# get adjp
parse = sent._.parse_string

In [12]:
split = parse.split()
print(split)

['(S', '(NP', '(NNP', 'US))', '(VP', '(VP', '(VBZ', 'unveils)', '(NP', '(NP', '(NN', 'world)', '(POS', "'s))", '(ADJP', '(RBS', 'most)', '(JJ', 'powerful))', '(NN', 'supercomputer)))', '(,', ',)', '(VP', '(VBZ', 'beats)', '(NP', '(NN', 'china)))))']


In [13]:
list(sent._.children)


[US, unveils world's most powerful supercomputer, beats china]

In [14]:
def traverse_children(children_list):
    adjp_list = []
    for child in children_list:
        adjp_found = False
        labels = child._.labels
        for label in labels:
            if (label.find('ADJP') != -1):
                adjp_list.append(child.text)
                adjp_found = True
                break
        
        if (not adjp_found):
            adjp_list.extend(traverse_children(list(child._.children)))
        else:
            result = traverse_children(list(child._.children))
            if (len(result) != 0):
                # means there is adjp inside, replace the with the inside one
                
    return adjp_list

def find_adjp_v2(sentence):
    sentence_doc = nlp(sentence)
    sent = list(sentence_doc.sents)[0]
    labels = sent._.labels
    adjp_found = False
    for label in labels:
        if (label.find('ADJP') != -1):
            adjp_found = True
            break
    if (adjp_found):
        return [sentence]
    
    return traverse_children(list(sent._.children))

In [15]:
find_adjp_v2("US unveils world's most powerful supercomputer, beats china")

['most powerful']

In [16]:
find_adjp_v2('so tidy and well-maintained')

['so tidy and well-maintained']

In [17]:
list(list(sent._.children)[1]._.children)

[unveils world's most powerful supercomputer, ,, beats china]

In [18]:
list(sent._.children)

[US, unveils world's most powerful supercomputer, beats china]

In [19]:
type(list(sent._.children)[0].text)

str

In [20]:
list(sent._.children)[0]._.labels

('NP',)

In [21]:
print(sent._.constituents)

<generator object get_subconstituents at 0x7fd7dff35820>


In [22]:
list(sent._.constituents)

[US unveils world's most powerful supercomputer, beats china,
 US,
 unveils world's most powerful supercomputer, beats china,
 unveils world's most powerful supercomputer,
 unveils,
 world's most powerful supercomputer,
 world's,
 world,
 's,
 most powerful,
 most,
 powerful,
 supercomputer,
 ,,
 beats china,
 beats,
 china]

In [23]:
list(sent._.constituents)[0]._.parse_string

"(S (NP (NNP US)) (VP (VP (VBZ unveils) (NP (NP (NN world) (POS 's)) (ADJP (RBS most) (JJ powerful)) (NN supercomputer))) (, ,) (VP (VBZ beats) (NP (NN china)))))"

In [24]:
type(list(sent._.constituents)[9]._.labels[0])

str

In [25]:
list(sent._.constituents)[9]

most powerful

In [26]:
def find_adjp_in_sentence(sentence):
    adjp_list = []
    sentence_doc = nlp(sentence)
    sent = list(sentence_doc.sents)[0]
    parse_tree_list = list(sent._.constituents)
    for i in range(len(parse_tree_list)):
        print(parse_tree_list[i])
        label_tuple = parse_tree_list[i]._.labels
        print('label tuple {}'.format(label_tuple))
        for label in label_tuple:
            if (label.find('ADJP') != -1):
                # is adjp, return
                adjp_list.append(list(sent._.constituents)[i])
    print('Number of adjp found: {}'.format(len(adjp_list)))
    return adjp_list

def process_text(df):
    text = df['text'].astype('str')
    adjp_dict = {}
    count = 0
    num_sentences = 0
    for t in text:
        if count == 20:
            break
        if count%10 == 0:
            print(count)
        count += 1
        
        doc = nlp(t)
        for sent in doc.sents:
            num_sentences += 1 
            adjp_list = find_adjp_v2(str(sent))
            
            # put the list into adjp dict
            for i in range(len(adjp_list)):
                key = adjp_list[i].lower()  # change everything to lower
                if key in adjp_dict:
                    adjp_dict[key] += 1
                else:
                    adjp_dict[key] = 1
    print('num sentences {}'.format(num_sentences))
    return adjp_dict

In [28]:
%%time
adjp_dict = process_text(df)

0
10
num sentences 148
CPU times: user 26.4 s, sys: 1.3 s, total: 27.7 s
Wall time: 28.3 s


In [97]:
adjp_dict

{'great': 1,
 'good': 2,
 'bland': 1,
 'meh': 1,
 'really flavourable': 1,
 'attentive, nice and considerate': 1,
 'way bigger': 1,
 'sure if the guy was the owner': 1,
 'friendly': 1,
 'so delicious': 1,
 'perfect': 2,
 'amazing': 4,
 'crunchy': 1,
 'worth the drive': 1,
 'clean': 1,
 'fast and friendly': 1,
 'delicious': 1,
 'this nice': 1,
 'nicest looking': 1,
 'terrible': 1,
 'very clear': 1,
 'more aggressive': 1,
 'unfortunate.': 1,
 'very good': 1,
 'a little salty on some items during our first visit': 1,
 'good overall and again': 1,
 'horrible': 1,
 'only able to contact them when i called them again': 1,
 'more than capable to answer my questions - about a pump that she has not seen, and about a conversation i had with the tech that she did not hear': 1,
 'cash out': 1,
 'able to find them on amazon': 1,
 'really good': 2,
 'sweet': 1,
 'really nice': 1,
 'soooo nice': 1,
 'very attentive': 1,
 'so good': 1,
 'hard': 1,
 'far superior to the reeses pancakes i had tried at o

In [ ]:
# using ntlk + standford core

In [4]:
# start server, dont run this, start server from terminal
# from nltk.parse.corenlp import CoreNLPServer
# import os

# # The server needs to know the location of the following files:
# #   - stanford-corenlp-X.X.X.jar
# #   - stanford-corenlp-X.X.X-models.jar
# STANFORD = os.path.join("models", "/Users/cecilialee/Downloads/stanford-corenlp-4.3.1")

# # Create the server
# server = CoreNLPServer(
#    os.path.join(STANFORD, "stanford-corenlp-4.3.1.jar"),
#    os.path.join(STANFORD, "stanford-corenlp-4.3.1-models.jar"),
# )

# # Start the server in the background
# server.start()



In [4]:
# define parser
from  nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')

In [192]:
def parse_using_standford(sentence_tuple):
    result_list = []
    sentences = parser.raw_parse_sents(sentence_tuple)
#     test = len(list(sentences)) cannot do this it will just die
#     print('is there return value {}'.format(test))
    for line in sentences:
#         print('whats happening... isit forever loop here ')
        for sentence in line:
#             str_sent = ' '.join(str(sentence).split())
#             result_list.extend(get_adjp(str_sent))
#             print(sentence)
            have_adjp, adjp_list = traverse_tree_sf(sentence)
            result_list.extend(adjp_list)
#     print('len of result list in stanford: {}'.format(len(result_list)))
    return result_list

In [194]:
def traverse_tree_sf(t):
    try:
        t.label()
    except AttributeError:
        return
    adjp_list = []
    subtree_have_adjp = False
    
#     print('this')
    for subtree in t:
        have_adjp = False
#         print(subtree.height())
#         print(subtree)
        if subtree.height() != 2:
#             print(subtree)
            have_adjp, adjp_list_subtree = traverse_tree_sf(subtree)
            
        if (have_adjp):
            subtree_have_adjp = True
            adjp_list.extend(adjp_list_subtree)
            
#         print('adjp list.. {}'.format(adjp_list))
#     print('current tree after recursion: {}'.format(t))
#     print('label: {}'.format(t.label()))
    if t.label().find("ADJP") != -1:
        if (subtree_have_adjp):
#             print('in subtree')
            return True, adjp_list
        else:
#             print('not in subtree')
#             print(t.leaves())
            return True, [" ".join(t.leaves())]
    else:
        return subtree_have_adjp, adjp_list
    
        

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cecilialee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [88]:
import re
def process_text_sf(df):
    text = df['text'].astype('str')
    adjp_dict = {}
    all_text = ''.join(map(str, text))
    all_text = re.sub(r'([a-z]).([A-Z])', r'\1. \2', all_text)
    sentences = sent_tokenize(all_text)
    print('number of sentences: {}'.format(len(sentences)))
    
    # loop in 100
    loop = 10
    for i in range(loop, len(sentences), loop):
        if (i % 100 == 0):
            print(i)
        
        adjp_list = parse_using_standford(tuple(sentences[i-loop:i]))

        # put the list into adjp dict
        for i in range(len(adjp_list)):
            key = adjp_list[i].lower()  # change everything to lower
            if key in adjp_dict:
                adjp_dict[key] += 1
            else:
                adjp_dict[key] = 1
    return adjp_dict

In [25]:
def connect():
    from  nltk.parse.corenlp import CoreNLPParser
    parser = CoreNLPParser(url='http://localhost:9000')
    print('reconnecting to standford core server ... ')
    return parser

In [127]:
import json
def process_text_sf_batch(df):
    text = df['text'].astype('str')
    adjp_dict = {}
    all_text = ''.join(map(str, text))
    all_text = re.sub(r'([a-z]).([A-Z])', r'\1. \2', all_text)
    sentences = sent_tokenize(all_text)
    print('number of sentences: {}'.format(len(sentences)))
    
    batch = 5000
    for i in range(batch, len(sentences)+1, batch):
        # print('len of dict: {}'.format(adjp_dict))
        print('batch {} from {}'.format(i/batch, i))
        adjp_dict = batches(tuple(sentences[i-batch:i]), adjp_dict)
        # save into text file
        with open('../result/run3/adjp_sf_batch_{}.json'.format(i), 'w') as file:
            file.write(json.dumps(adjp_dict)) # use `json.loads` to do the reverse

        # reconnct again
        parser = connect()
    return adjp_dict

In [130]:
def batches(sentences_tuple, adjp_dict):
  # print('lens {}'.format(len(sentences_tuple)))
    loop = 10
    for i in range(loop, len(sentences_tuple)+1, loop):
    # put the list into adjp dict
#         if i == 196:
#             continue
        
        print('loop in batch {}'.format(i))
#         print('len of sentence: {}'.format(len(sentences_tuple[i-loop])))
#         print('len of sentence: {}'.format(len(sentences_tuple[i-loop+1])))

        adjp_list = parse_using_standford(sentences_tuple[i-loop:i])
        for r in range(len(adjp_list)):
            key = adjp_list[r].lower()  # change everything to lower
            if key in adjp_dict:
                adjp_dict[key] += 1
            else:
                adjp_dict[key] = 1
    return adjp_dict

In [185]:
with open('../result/run3/adjp_sf_batch_130000.json') as json_file:
    previous_dict = json.load(json_file)

In [186]:
len(previous_dict)

0

In [129]:
%%time
parse_dict = process_text_sf_batch(df)

number of sentences: 192796
batch 1.0 from 5000
loop in batch 10
loop in batch 20
loop in batch 30
loop in batch 40
loop in batch 50
loop in batch 60
loop in batch 70
loop in batch 80
loop in batch 90
loop in batch 100
loop in batch 110
loop in batch 120
loop in batch 130
loop in batch 140
loop in batch 150
loop in batch 160
loop in batch 170
loop in batch 180
loop in batch 190
loop in batch 200
loop in batch 210
loop in batch 220
loop in batch 230
loop in batch 240
loop in batch 250
loop in batch 260
loop in batch 270
loop in batch 280
loop in batch 290
loop in batch 300
loop in batch 310
loop in batch 320
loop in batch 330
loop in batch 340
loop in batch 350
loop in batch 360
loop in batch 370
loop in batch 380
loop in batch 390
loop in batch 400
loop in batch 410
loop in batch 420
loop in batch 430
loop in batch 440
loop in batch 450
loop in batch 460
loop in batch 470
loop in batch 480
loop in batch 490
loop in batch 500
loop in batch 510
loop in batch 520
loop in batch 530
loop in

loop in batch 4360
loop in batch 4370
loop in batch 4380
loop in batch 4390
loop in batch 4400
loop in batch 4410
loop in batch 4420
loop in batch 4430
loop in batch 4440
loop in batch 4450
loop in batch 4460
loop in batch 4470
loop in batch 4480
loop in batch 4490
loop in batch 4500
loop in batch 4510
loop in batch 4520
loop in batch 4530
loop in batch 4540
loop in batch 4550
loop in batch 4560
loop in batch 4570
loop in batch 4580
loop in batch 4590
loop in batch 4600
loop in batch 4610
loop in batch 4620
loop in batch 4630
loop in batch 4640
loop in batch 4650
loop in batch 4660
loop in batch 4670
loop in batch 4680
loop in batch 4690
loop in batch 4700
loop in batch 4710
loop in batch 4720
loop in batch 4730
loop in batch 4740
loop in batch 4750
loop in batch 4760
loop in batch 4770
loop in batch 4780
loop in batch 4790
loop in batch 4800
loop in batch 4810
loop in batch 4820
loop in batch 4830
loop in batch 4840
loop in batch 4850
loop in batch 4860
loop in batch 4870
loop in batc

loop in batch 3710
loop in batch 3720
loop in batch 3730
loop in batch 3740
loop in batch 3750
loop in batch 3760
loop in batch 3770
loop in batch 3780
loop in batch 3790
loop in batch 3800
loop in batch 3810
loop in batch 3820
loop in batch 3830
loop in batch 3840
loop in batch 3850
loop in batch 3860
loop in batch 3870
loop in batch 3880
loop in batch 3890
loop in batch 3900
loop in batch 3910
loop in batch 3920
loop in batch 3930
loop in batch 3940
loop in batch 3950
loop in batch 3960
loop in batch 3970
loop in batch 3980
loop in batch 3990
loop in batch 4000
loop in batch 4010
loop in batch 4020
loop in batch 4030
loop in batch 4040
loop in batch 4050
loop in batch 4060
loop in batch 4070
loop in batch 4080
loop in batch 4090
loop in batch 4100
loop in batch 4110
loop in batch 4120
loop in batch 4130
loop in batch 4140
loop in batch 4150
loop in batch 4160
loop in batch 4170
loop in batch 4180
loop in batch 4190
loop in batch 4200
loop in batch 4210
loop in batch 4220
loop in batc

loop in batch 3050
loop in batch 3060
loop in batch 3070
loop in batch 3080
loop in batch 3090
loop in batch 3100
loop in batch 3110
loop in batch 3120
loop in batch 3130
loop in batch 3140
loop in batch 3150
loop in batch 3160
loop in batch 3170
loop in batch 3180
loop in batch 3190
loop in batch 3200
loop in batch 3210
loop in batch 3220
loop in batch 3230
loop in batch 3240
loop in batch 3250
loop in batch 3260
loop in batch 3270
loop in batch 3280
loop in batch 3290
loop in batch 3300
loop in batch 3310
loop in batch 3320
loop in batch 3330
loop in batch 3340
loop in batch 3350
loop in batch 3360
loop in batch 3370
loop in batch 3380
loop in batch 3390
loop in batch 3400
loop in batch 3410
loop in batch 3420
loop in batch 3430
loop in batch 3440
loop in batch 3450
loop in batch 3460
loop in batch 3470
loop in batch 3480
loop in batch 3490
loop in batch 3500
loop in batch 3510
loop in batch 3520
loop in batch 3530
loop in batch 3540
loop in batch 3550
loop in batch 3560
loop in batc

loop in batch 2390
loop in batch 2400
loop in batch 2410
loop in batch 2420
loop in batch 2430
loop in batch 2440
loop in batch 2450
loop in batch 2460
loop in batch 2470
loop in batch 2480
loop in batch 2490
loop in batch 2500
loop in batch 2510
loop in batch 2520
loop in batch 2530
loop in batch 2540
loop in batch 2550
loop in batch 2560
loop in batch 2570
loop in batch 2580
loop in batch 2590
loop in batch 2600
loop in batch 2610
loop in batch 2620
loop in batch 2630
loop in batch 2640
loop in batch 2650
loop in batch 2660
loop in batch 2670
loop in batch 2680
loop in batch 2690
loop in batch 2700
loop in batch 2710
loop in batch 2720
loop in batch 2730
loop in batch 2740
loop in batch 2750
loop in batch 2760
loop in batch 2770
loop in batch 2780
loop in batch 2790
loop in batch 2800
loop in batch 2810
loop in batch 2820
loop in batch 2830
loop in batch 2840
loop in batch 2850
loop in batch 2860
loop in batch 2870
loop in batch 2880
loop in batch 2890
loop in batch 2900
loop in batc

...

loop in batch 2200
loop in batch 2210
loop in batch 2220
loop in batch 2230
loop in batch 2240
loop in batch 2250
loop in batch 2260
loop in batch 2270
loop in batch 2280
loop in batch 2290
loop in batch 2300
loop in batch 2310
loop in batch 2320
loop in batch 2330
loop in batch 2340
loop in batch 2350
loop in batch 2360
loop in batch 2370
loop in batch 2380
loop in batch 2390
loop in batch 2400
loop in batch 2410
loop in batch 2420
loop in batch 2430
loop in batch 2440
loop in batch 2450
loop in batch 2460
loop in batch 2470
loop in batch 2480
loop in batch 2490
loop in batch 2500
loop in batch 2510
loop in batch 2520
loop in batch 2530
loop in batch 2540
loop in batch 2550
loop in batch 2560
loop in batch 2570
loop in batch 2580
loop in batch 2590
loop in batch 2600
loop in batch 2610
loop in batch 2620
loop in batch 2630
loop in batch 2640
loop in batch 2650
loop in batch 2660
loop in batch 2670
loop in batch 2680
loop in batch 2690
loop in batch 2700
loop in batch 2710
loop in batc

loop in batch 1540
loop in batch 1550
loop in batch 1560
loop in batch 1570
loop in batch 1580
loop in batch 1590
loop in batch 1600
loop in batch 1610
loop in batch 1620
loop in batch 1630
loop in batch 1640
loop in batch 1650
loop in batch 1660
loop in batch 1670
loop in batch 1680
loop in batch 1690
loop in batch 1700
loop in batch 1710
loop in batch 1720
loop in batch 1730
loop in batch 1740
loop in batch 1750
loop in batch 1760
loop in batch 1770
loop in batch 1780
loop in batch 1790
loop in batch 1800
loop in batch 1810
loop in batch 1820
loop in batch 1830
loop in batch 1840
loop in batch 1850
loop in batch 1860
loop in batch 1870
loop in batch 1880
loop in batch 1890
loop in batch 1900
loop in batch 1910
loop in batch 1920
loop in batch 1930
loop in batch 1940
loop in batch 1950
loop in batch 1960
loop in batch 1970
loop in batch 1980
loop in batch 1990
loop in batch 2000
loop in batch 2010
loop in batch 2020
loop in batch 2030
loop in batch 2040
loop in batch 2050
loop in batc

loop in batch 870
loop in batch 880
loop in batch 890
loop in batch 900
loop in batch 910
loop in batch 920
loop in batch 930
loop in batch 940
loop in batch 950
loop in batch 960
loop in batch 970
loop in batch 980
loop in batch 990
loop in batch 1000
loop in batch 1010
loop in batch 1020
loop in batch 1030
loop in batch 1040
loop in batch 1050
loop in batch 1060
loop in batch 1070
loop in batch 1080
loop in batch 1090
loop in batch 1100
loop in batch 1110
loop in batch 1120
loop in batch 1130
loop in batch 1140
loop in batch 1150
loop in batch 1160
loop in batch 1170
loop in batch 1180
loop in batch 1190
loop in batch 1200
loop in batch 1210
loop in batch 1220
loop in batch 1230
loop in batch 1240
loop in batch 1250
loop in batch 1260
loop in batch 1270
loop in batch 1280
loop in batch 1290
loop in batch 1300
loop in batch 1310
loop in batch 1320
loop in batch 1330
loop in batch 1340
loop in batch 1350
loop in batch 1360
loop in batch 1370
loop in batch 1380
loop in batch 1390
loop i

loop in batch 170
loop in batch 180
loop in batch 190
loop in batch 200
loop in batch 210
loop in batch 220
loop in batch 230
loop in batch 240
loop in batch 250
loop in batch 260
loop in batch 270
loop in batch 280
loop in batch 290
loop in batch 300
loop in batch 310
loop in batch 320
loop in batch 330
loop in batch 340
loop in batch 350
loop in batch 360
loop in batch 370
loop in batch 380
loop in batch 390
loop in batch 400
loop in batch 410
loop in batch 420
loop in batch 430
loop in batch 440
loop in batch 450
loop in batch 460
loop in batch 470
loop in batch 480
loop in batch 490
loop in batch 500
loop in batch 510
loop in batch 520
loop in batch 530
loop in batch 540
loop in batch 550
loop in batch 560
loop in batch 570
loop in batch 580
loop in batch 590
loop in batch 600
loop in batch 610
loop in batch 620
loop in batch 630
loop in batch 640
loop in batch 650
loop in batch 660
loop in batch 670
loop in batch 680
loop in batch 690
loop in batch 700
loop in batch 710
loop in ba

ReadTimeout: HTTPConnectionPool(host='localhost', port=9000): Read timed out. (read timeout=60)

In [ ]:
# parse into sentence by spacy, then use stanford core nlp for consituency parsing
# nltk split sentence sucks ..

In [131]:
def process_text_sf_batch_from_prev(df, adjp_dict):
    text = df['text'].astype('str')
    all_text = ''.join(map(str, text))
    all_text = re.sub(r'([a-z]).([A-Z])', r'\1. \2', all_text)
    sentences = sent_tokenize(all_text)
    
    
    print('number of sentences: {}'.format(len(sentences)))
    
    batch = 5000
    for i in range(65000, len(sentences)+1, batch):
        # print('len of dict: {}'.format(adjp_dict))
        print('batch {} from {}'.format(i/batch, i))
        adjp_dict = batches(tuple(sentences[i-batch:i]), adjp_dict)
        # save into text file
        with open('../result/run2/adjp_sf_batch_{}.json'.format(i), 'w') as file:
            file.write(json.dumps(adjp_dict)) # use `json.loads` to do the reverse

        # reconnct again
        parser = connect()
    return adjp_dict

In [207]:

def process_text_split_sentence_sf(df):
    # use spacy to split sentences
    nlp = spacy.load('en_core_web_sm')
    text = df['text'].astype('str')
    count = 0
    num_sentences = 0
    print('Number of text {}'.format(len(text)))
    adjp_dict={}
    batch = 500
    for i in range(batch, len(text)+1, batch):
        print('batch {} from {}'.format(i/batch, i))
        for r in range(batch):
            print('text id {}'.format(i-batch+r))
            doc = nlp(text[i-batch+r])
            generator = list(doc.sents)
            sentence_tuple = [x.text for x in doc.sents]
            sentence_tuple = tuple(sentence_tuple)
#             print(sentence_tuple)
#             print('sentence tuple len {}'.format(len(sentence_tuple)))
#             adjp_dict = batches(sentence_tuple, adjp_dict)
            
            adjp_list = parse_using_standford(sentence_tuple)
#             print('len of adjp list {}'.format(adjp_list))
            # put the list into adjp dict
            for x in range(len(adjp_list)):
                key = adjp_list[x].lower()  # change everything to lower
                if key in adjp_dict:
                    adjp_dict[key] += 1
                else:
                    adjp_dict[key] = 1
        
        # save into json
        with open('../result/run4/adjp_sf_batch_{}.json'.format(i), 'w') as file:
            file.write(json.dumps(adjp_dict)) # use `json.loads` to do the reverse
            
        # connect to server again
        parser = connect()
    return adjp_dict

In [195]:
test = ['Good Korean grill near Eaton Centre.', 'The marinate is good.', 'We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly.', 'The fish fillet was bland and liver was meh.', 'Salmon and chicken was really flavourable.', 'Such a fun place to eat at for a date or group of friends.', 'Even alone.', 'No judgments here.', '\n', 'The staff is attentive, nice and considerate.', 'Bigger groups will most likely be seated on the second floor which is way bigger.', '\nCaution: will smell like BBQ grill after.']
adjp_dict = {}
adjp_dict = batches(tuple(test), adjp_dict)
adjp_dict

loop in batch 10


{'good': 1,
 'bland': 1,
 'really flavourable': 1,
 'even alone .': 1,
 'attentive , nice and considerate': 1}

In [ ]:
%%time
parse_dict_test = process_text_split_sentence_sf(df)

Number of text 15300
batch 1.0 from 500
text id 0
text id 1
text id 2
text id 3
text id 4
text id 5
text id 6
text id 7
text id 8
text id 9
text id 10
text id 11
text id 12
text id 13
text id 14
text id 15
text id 16
text id 17
text id 18
text id 19
text id 20
text id 21
text id 22
text id 23
text id 24
text id 25
text id 26
text id 27
text id 28
text id 29
text id 30
text id 31
text id 32
text id 33
text id 34
text id 35
text id 36
text id 37
text id 38
text id 39
text id 40
text id 41
text id 42
text id 43
text id 44
text id 45
text id 46
text id 47
text id 48
text id 49
text id 50
text id 51
text id 52
text id 53
text id 54
text id 55
text id 56
text id 57
text id 58
text id 59
text id 60
text id 61
text id 62
text id 63
text id 64
text id 65
text id 66
text id 67
text id 68
text id 69
text id 70
text id 71
text id 72
text id 73
text id 74
text id 75
text id 76
text id 77
text id 78
text id 79
text id 80
text id 81
text id 82
text id 83
text id 84
text id 85
text id 86
text id 87
te

text id 685
text id 686
text id 687
text id 688
text id 689
text id 690
text id 691
text id 692
text id 693
text id 694
text id 695
text id 696
text id 697
text id 698
text id 699
text id 700
text id 701
text id 702
text id 703
text id 704
text id 705
text id 706
text id 707
text id 708
text id 709
text id 710
text id 711
text id 712
text id 713
text id 714
text id 715
text id 716
text id 717
text id 718
text id 719
text id 720
text id 721
text id 722
text id 723
text id 724
text id 725
text id 726
text id 727
text id 728
text id 729
text id 730
text id 731
text id 732
text id 733
text id 734
text id 735
text id 736
text id 737
text id 738
text id 739
text id 740
text id 741
text id 742
text id 743
text id 744
text id 745
text id 746
text id 747
text id 748
text id 749
text id 750
text id 751
text id 752
text id 753
text id 754
text id 755
text id 756
text id 757
text id 758
text id 759
text id 760
text id 761
text id 762
text id 763
text id 764
text id 765
text id 766
text id 767
text

text id 1336
text id 1337
text id 1338
text id 1339
text id 1340
text id 1341
text id 1342
text id 1343
text id 1344
text id 1345
text id 1346
text id 1347
text id 1348
text id 1349
text id 1350
text id 1351
text id 1352
text id 1353
text id 1354
text id 1355
text id 1356
text id 1357
text id 1358
text id 1359
text id 1360
text id 1361
text id 1362
text id 1363
text id 1364
text id 1365
text id 1366
text id 1367
text id 1368
text id 1369
text id 1370
text id 1371
text id 1372
text id 1373
text id 1374
text id 1375
text id 1376
text id 1377
text id 1378
text id 1379
text id 1380
text id 1381
text id 1382
text id 1383
text id 1384
text id 1385
text id 1386
text id 1387
text id 1388
text id 1389
text id 1390
text id 1391
text id 1392
text id 1393
text id 1394
text id 1395
text id 1396
text id 1397
text id 1398
text id 1399
text id 1400
text id 1401
text id 1402
text id 1403
text id 1404
text id 1405
text id 1406
text id 1407
text id 1408
text id 1409
text id 1410
text id 1411
text id 1412

text id 1962
text id 1963
text id 1964
text id 1965
text id 1966
text id 1967
text id 1968
text id 1969
text id 1970
text id 1971
text id 1972
text id 1973
text id 1974
text id 1975
text id 1976
text id 1977
text id 1978
text id 1979
text id 1980
text id 1981
text id 1982
text id 1983
text id 1984
text id 1985
text id 1986
text id 1987
text id 1988
text id 1989
text id 1990
text id 1991
text id 1992
text id 1993
text id 1994
text id 1995
text id 1996
text id 1997
text id 1998
text id 1999
reconnecting to standford core server ... 
batch 5.0 from 2500
text id 2000
text id 2001
text id 2002
text id 2003
text id 2004
text id 2005
text id 2006
text id 2007
text id 2008
text id 2009
text id 2010
text id 2011
text id 2012
text id 2013
text id 2014
text id 2015
text id 2016
text id 2017
text id 2018
text id 2019
text id 2020
text id 2021
text id 2022
text id 2023
text id 2024
text id 2025
text id 2026
text id 2027
text id 2028
text id 2029
text id 2030
text id 2031
text id 2032
text id 2033
t

text id 2584
text id 2585
text id 2586
text id 2587
text id 2588
text id 2589
text id 2590
text id 2591
text id 2592
text id 2593
text id 2594
text id 2595
text id 2596
text id 2597
text id 2598
text id 2599
text id 2600
text id 2601
text id 2602
text id 2603
text id 2604
text id 2605
text id 2606
text id 2607
text id 2608
text id 2609
text id 2610
text id 2611
text id 2612
text id 2613
text id 2614
text id 2615
text id 2616
text id 2617
text id 2618
text id 2619
text id 2620
text id 2621
text id 2622
text id 2623
text id 2624
text id 2625
text id 2626
text id 2627
text id 2628
text id 2629
text id 2630
text id 2631
text id 2632
text id 2633
text id 2634
text id 2635
text id 2636
text id 2637
text id 2638
text id 2639
text id 2640
text id 2641
text id 2642
text id 2643
text id 2644
text id 2645
text id 2646
text id 2647
text id 2648
text id 2649
text id 2650
text id 2651
text id 2652
text id 2653
text id 2654
text id 2655
text id 2656
text id 2657
text id 2658
text id 2659
text id 2660

...

text id 12404
text id 12405
text id 12406
text id 12407
text id 12408
text id 12409
text id 12410
text id 12411
text id 12412
text id 12413
text id 12414
text id 12415
text id 12416
text id 12417
text id 12418
text id 12419
text id 12420
text id 12421
text id 12422
text id 12423
text id 12424
text id 12425
text id 12426
text id 12427
text id 12428
text id 12429
text id 12430
text id 12431
text id 12432
text id 12433
text id 12434
text id 12435
text id 12436
text id 12437
text id 12438
text id 12439
text id 12440
text id 12441
text id 12442
text id 12443
text id 12444
text id 12445
text id 12446
text id 12447
text id 12448
text id 12449
text id 12450
text id 12451
text id 12452
text id 12453
text id 12454
text id 12455
text id 12456
text id 12457
text id 12458
text id 12459
text id 12460
text id 12461
text id 12462
text id 12463
text id 12464
text id 12465
text id 12466
text id 12467
text id 12468
text id 12469
text id 12470
text id 12471
text id 12472
text id 12473
text id 12474
text i

text id 12986
text id 12987
text id 12988
text id 12989
text id 12990
text id 12991
text id 12992
text id 12993
text id 12994
text id 12995
text id 12996
text id 12997
text id 12998
text id 12999
reconnecting to standford core server ... 
batch 27.0 from 13500
text id 13000
text id 13001
text id 13002
text id 13003
text id 13004
text id 13005
text id 13006
text id 13007
text id 13008
text id 13009
text id 13010
text id 13011
text id 13012
text id 13013
text id 13014
text id 13015
text id 13016
text id 13017
text id 13018
text id 13019
text id 13020
text id 13021
text id 13022
text id 13023
text id 13024
text id 13025
text id 13026
text id 13027
text id 13028
text id 13029
text id 13030
text id 13031
text id 13032
text id 13033
text id 13034
text id 13035
text id 13036
text id 13037
text id 13038
text id 13039
text id 13040
text id 13041
text id 13042
text id 13043
text id 13044
text id 13045
text id 13046
text id 13047
text id 13048
text id 13049
text id 13050
text id 13051
text id 130

text id 13562
text id 13563
text id 13564
text id 13565
text id 13566
text id 13567
text id 13568
text id 13569
text id 13570
text id 13571
text id 13572
text id 13573
text id 13574
text id 13575
text id 13576
text id 13577
text id 13578
text id 13579
text id 13580
text id 13581
text id 13582
text id 13583
text id 13584
text id 13585
text id 13586
text id 13587
text id 13588
text id 13589
text id 13590
text id 13591
text id 13592
text id 13593
text id 13594
text id 13595
text id 13596
text id 13597
text id 13598
text id 13599
text id 13600
text id 13601
text id 13602
text id 13603
text id 13604
text id 13605
text id 13606
text id 13607
text id 13608
text id 13609
text id 13610
text id 13611
text id 13612
text id 13613
text id 13614
text id 13615
text id 13616
text id 13617
text id 13618
text id 13619
text id 13620
text id 13621
text id 13622
text id 13623
text id 13624
text id 13625
text id 13626
text id 13627
text id 13628
text id 13629
text id 13630
text id 13631
text id 13632
text i

text id 14145
text id 14146
text id 14147
text id 14148
text id 14149
text id 14150
text id 14151
text id 14152
text id 14153
text id 14154
text id 14155
text id 14156
text id 14157
text id 14158
text id 14159
text id 14160
text id 14161
text id 14162
text id 14163
text id 14164
text id 14165
text id 14166
text id 14167
text id 14168
text id 14169
text id 14170
text id 14171
text id 14172
text id 14173
text id 14174
text id 14175
text id 14176
text id 14177
text id 14178
text id 14179
text id 14180
text id 14181
text id 14182
text id 14183
text id 14184
text id 14185
text id 14186
text id 14187
text id 14188
text id 14189
text id 14190
text id 14191
text id 14192
text id 14193
text id 14194
text id 14195
text id 14196
text id 14197
text id 14198
text id 14199
text id 14200
text id 14201
text id 14202
text id 14203
text id 14204
text id 14205
text id 14206
text id 14207
text id 14208
text id 14209
text id 14210
text id 14211
text id 14212
text id 14213
text id 14214
text id 14215
text i

text id 14727
text id 14728
text id 14729
text id 14730
text id 14731
text id 14732
text id 14733
text id 14734
text id 14735
text id 14736
text id 14737
text id 14738
text id 14739
text id 14740
text id 14741
text id 14742
text id 14743
text id 14744
text id 14745
text id 14746
text id 14747
text id 14748
text id 14749
text id 14750
text id 14751
text id 14752
text id 14753
text id 14754
text id 14755
text id 14756
text id 14757
text id 14758
text id 14759
text id 14760
text id 14761
text id 14762
text id 14763
text id 14764
text id 14765
text id 14766
text id 14767
text id 14768
text id 14769
text id 14770
text id 14771
text id 14772
text id 14773
text id 14774
text id 14775
text id 14776
text id 14777
text id 14778
text id 14779
text id 14780
text id 14781
text id 14782
text id 14783
text id 14784
text id 14785
text id 14786
text id 14787
text id 14788
text id 14789
text id 14790
text id 14791
text id 14792
text id 14793
text id 14794
text id 14795
text id 14796
text id 14797
text i

In [205]:
parse_dict_test

{'great': 5,
 'good': 4,
 'bland': 1,
 'really flavourable': 1,
 'even alone .': 1,
 'attentive , nice and considerate': 1,
 'way bigger': 1,
 'sure': 1,
 'friendly': 1,
 'so delicious': 1,
 'perfect': 3,
 'amazing': 4,
 'crunchy': 1,
 'clean': 1,
 'fast and friendly': 1,
 'delicious': 2,
 'what more': 1,
 'nice at cannery hotel': 1,
 'terrible': 1,
 'fairly close': 1,
 'very clear': 1,
 'more aggressive': 1,
 'unfortunate .': 1,
 'very good': 2,
 'a little salty': 1,
 'horrible': 1,
 'able to contact them when i called them again': 1,
 'more than capable to answer my questions - about a pump that she has not seen , and about a conversation i had with the tech that she did not hear': 1,
 'able to find them on amazon': 1,
 'really good': 2,
 'sweet': 2,
 "really nice inside it does n't seem like your at the cannery": 1,
 'nice': 2,
 'very attentive': 2,
 'so good': 2,
 'fairly quick': 1,
 'finished': 1,
 'far superior to the reeses pancakes': 1,
 'higher': 1,
 'better': 2,
 'interested'

In [41]:
# load from previous
with open('../result/run2/adjp_sf_batch_15000.json') as json_file:
    previous_dict = json.load(json_file)

In [42]:
previous_dict

{'great': 203,
 'good': 275,
 'bland': 16,
 'really flavourable': 2,
 'even alone .': 2,
 'attentive , nice and considerate': 2,
 'way bigger': 5,
 'sure': 60,
 'friendly': 59,
 'so delicious': 8,
 'perfect': 47,
 'amazing': 102,
 'crunchy': 7,
 'clean': 30,
 'fast and friendly': 4,
 'delicious': 95,
 'terrible': 27,
 'fairly close': 1,
 'very clear': 3,
 'more aggressive': 1,
 'unfortunate .': 1,
 'very good': 78,
 'a little salty': 2,
 'horrible': 24,
 'able to contact them when i called them again': 1,
 'more than capable to answer my questions - about a pump that she has not seen , and about a conversation i had with the tech that she did not hear': 1,
 'able to find them on amazon': 1,
 'really good': 55,
 'sweet': 10,
 "really nice inside it does n't seem like your at the cannery": 1,
 'nice': 56,
 'very attentive': 13,
 'so good': 25,
 'fairly quick': 3,
 'far superior to the reeses': 1,
 'higher': 2,
 'better': 63,
 'not interested': 1,
 'polite': 5,
 'forgettable , but good': 

In [43]:
len(previous_dict)

4918

In [123]:
def process_text_sf_batch_from_prev(df, adjp_dict):
    text = df['text'].astype('str')
    all_text = ''.join(map(str, text))
    all_text = re.sub(r'([a-z]).([A-Z])', r'\1. \2', all_text)
    sentences = sent_tokenize(all_text)
    print('number of sentences: {}'.format(len(sentences)))
    
    batch = 5000
    for i in range(65000, len(sentences)+1, batch):
        # print('len of dict: {}'.format(adjp_dict))
        print('batch {} from {}'.format(i/batch, i))
        adjp_dict = batches(tuple(sentences[i-batch:i]), adjp_dict)
        # save into text file
        with open('../result/run2/adjp_sf_batch_{}.json'.format(i), 'w') as file:
            file.write(json.dumps(adjp_dict)) # use `json.loads` to do the reverse

        # reconnct again
        parser = connect()
    return adjp_dict

In [45]:
# from 20000
process_text_sf_batch_from_prev(df, previous_dict)

number of sentences: 108953
batch 4.0 from 20000
loop in batch 5
loop in batch 10
loop in batch 15
loop in batch 20
loop in batch 25
loop in batch 30
loop in batch 35
loop in batch 40
loop in batch 45
loop in batch 50
loop in batch 55
loop in batch 60
loop in batch 65
loop in batch 70
loop in batch 75
loop in batch 80
loop in batch 85
loop in batch 90
loop in batch 95
loop in batch 100
loop in batch 105
loop in batch 110
loop in batch 115
loop in batch 120
loop in batch 125
loop in batch 130
loop in batch 135
loop in batch 140
loop in batch 145
loop in batch 150
loop in batch 155
loop in batch 160
loop in batch 165
loop in batch 170
loop in batch 175
loop in batch 180
loop in batch 185
loop in batch 190
loop in batch 195
loop in batch 200
loop in batch 205
loop in batch 210
loop in batch 215
loop in batch 220
loop in batch 225
loop in batch 230
loop in batch 235
loop in batch 240
loop in batch 245
loop in batch 250
loop in batch 255
loop in batch 260
loop in batch 265
loop in batch 270

loop in batch 2215
loop in batch 2220
loop in batch 2225
loop in batch 2230
loop in batch 2235
loop in batch 2240
loop in batch 2245
loop in batch 2250
loop in batch 2255
loop in batch 2260
loop in batch 2265
loop in batch 2270
loop in batch 2275
loop in batch 2280
loop in batch 2285
loop in batch 2290
loop in batch 2295
loop in batch 2300
loop in batch 2305
loop in batch 2310
loop in batch 2315
loop in batch 2320
loop in batch 2325
loop in batch 2330
loop in batch 2335
loop in batch 2340
loop in batch 2345
loop in batch 2350
loop in batch 2355
loop in batch 2360
loop in batch 2365
loop in batch 2370
loop in batch 2375
loop in batch 2380
loop in batch 2385
loop in batch 2390
loop in batch 2395
loop in batch 2400
loop in batch 2405
loop in batch 2410
loop in batch 2415
loop in batch 2420
loop in batch 2425
loop in batch 2430
loop in batch 2435
loop in batch 2440
loop in batch 2445
loop in batch 2450
loop in batch 2455
loop in batch 2460
loop in batch 2465
loop in batch 2470
loop in batc

loop in batch 4375
loop in batch 4380
loop in batch 4385
loop in batch 4390
loop in batch 4395
loop in batch 4400
loop in batch 4405
loop in batch 4410
loop in batch 4415
loop in batch 4420
loop in batch 4425
loop in batch 4430
loop in batch 4435
loop in batch 4440
loop in batch 4445
loop in batch 4450
loop in batch 4455
loop in batch 4460
loop in batch 4465
loop in batch 4470
loop in batch 4475
loop in batch 4480
loop in batch 4485
loop in batch 4490
loop in batch 4495
loop in batch 4500
loop in batch 4505
loop in batch 4510
loop in batch 4515
loop in batch 4520
loop in batch 4525
loop in batch 4530
loop in batch 4535
loop in batch 4540
loop in batch 4545
loop in batch 4550
loop in batch 4555
loop in batch 4560
loop in batch 4565
loop in batch 4570
loop in batch 4575
loop in batch 4580
loop in batch 4585
loop in batch 4590
loop in batch 4595
loop in batch 4600
loop in batch 4605
loop in batch 4610
loop in batch 4615
loop in batch 4620
loop in batch 4625
loop in batch 4630
loop in batc

loop in batch 1575
loop in batch 1580
loop in batch 1585
loop in batch 1590
loop in batch 1595
loop in batch 1600
loop in batch 1605
loop in batch 1610
loop in batch 1615
loop in batch 1620
loop in batch 1625
loop in batch 1630
loop in batch 1635
loop in batch 1640
loop in batch 1645
loop in batch 1650
loop in batch 1655
loop in batch 1660
loop in batch 1665
loop in batch 1670
loop in batch 1675
loop in batch 1680
loop in batch 1685
loop in batch 1690
loop in batch 1695
loop in batch 1700
loop in batch 1705
loop in batch 1710
loop in batch 1715
loop in batch 1720
loop in batch 1725
loop in batch 1730
loop in batch 1735
loop in batch 1740
loop in batch 1745
loop in batch 1750
loop in batch 1755
loop in batch 1760
loop in batch 1765
loop in batch 1770
loop in batch 1775
loop in batch 1780
loop in batch 1785
loop in batch 1790
loop in batch 1795
loop in batch 1800
loop in batch 1805
loop in batch 1810
loop in batch 1815
loop in batch 1820
loop in batch 1825
loop in batch 1830
loop in batc

loop in batch 3735
loop in batch 3740
loop in batch 3745
loop in batch 3750
loop in batch 3755
loop in batch 3760
loop in batch 3765
loop in batch 3770
loop in batch 3775
loop in batch 3780
loop in batch 3785
loop in batch 3790
loop in batch 3795
loop in batch 3800
loop in batch 3805
loop in batch 3810
loop in batch 3815
loop in batch 3820
loop in batch 3825
loop in batch 3830
loop in batch 3835
loop in batch 3840
loop in batch 3845
loop in batch 3850
loop in batch 3855
loop in batch 3860
loop in batch 3865
loop in batch 3870
loop in batch 3875
loop in batch 3880
loop in batch 3885
loop in batch 3890
loop in batch 3895
loop in batch 3900
loop in batch 3905
loop in batch 3910
loop in batch 3915
loop in batch 3920
loop in batch 3925
loop in batch 3930
loop in batch 3935
loop in batch 3940
loop in batch 3945
loop in batch 3950
loop in batch 3955
loop in batch 3960
loop in batch 3965
loop in batch 3970
loop in batch 3975
loop in batch 3980
loop in batch 3985
loop in batch 3990
loop in batc

...

loop in batch 2055
loop in batch 2060
loop in batch 2065
loop in batch 2070
loop in batch 2075
loop in batch 2080
loop in batch 2085
loop in batch 2090
loop in batch 2095
loop in batch 2100
loop in batch 2105
loop in batch 2110
loop in batch 2115
loop in batch 2120
loop in batch 2125
loop in batch 2130
loop in batch 2135
loop in batch 2140
loop in batch 2145
loop in batch 2150
loop in batch 2155
loop in batch 2160
loop in batch 2165
loop in batch 2170
loop in batch 2175
loop in batch 2180
loop in batch 2185
loop in batch 2190
loop in batch 2195
loop in batch 2200
loop in batch 2205
loop in batch 2210
loop in batch 2215
loop in batch 2220
loop in batch 2225
loop in batch 2230
loop in batch 2235
loop in batch 2240
loop in batch 2245
loop in batch 2250
loop in batch 2255
loop in batch 2260
loop in batch 2265
loop in batch 2270
loop in batch 2275
loop in batch 2280
loop in batch 2285
loop in batch 2290
loop in batch 2295
loop in batch 2300
loop in batch 2305
loop in batch 2310
loop in batc

loop in batch 4215
loop in batch 4220
loop in batch 4225
loop in batch 4230
loop in batch 4235
loop in batch 4240
loop in batch 4245
loop in batch 4250
loop in batch 4255
loop in batch 4260
loop in batch 4265
loop in batch 4270
loop in batch 4275
loop in batch 4280
loop in batch 4285
loop in batch 4290
loop in batch 4295
loop in batch 4300
loop in batch 4305
loop in batch 4310
loop in batch 4315
loop in batch 4320
loop in batch 4325
loop in batch 4330
loop in batch 4335
loop in batch 4340
loop in batch 4345
loop in batch 4350
loop in batch 4355
loop in batch 4360
loop in batch 4365
loop in batch 4370
loop in batch 4375
loop in batch 4380
loop in batch 4385
loop in batch 4390
loop in batch 4395
loop in batch 4400
loop in batch 4405
loop in batch 4410
loop in batch 4415
loop in batch 4420
loop in batch 4425
loop in batch 4430
loop in batch 4435
loop in batch 4440
loop in batch 4445
loop in batch 4450
loop in batch 4455
loop in batch 4460
loop in batch 4465
loop in batch 4470
loop in batc

loop in batch 1415
loop in batch 1420
loop in batch 1425
loop in batch 1430
loop in batch 1435
loop in batch 1440
loop in batch 1445
loop in batch 1450
loop in batch 1455
loop in batch 1460
loop in batch 1465
loop in batch 1470
loop in batch 1475
loop in batch 1480
loop in batch 1485
loop in batch 1490
loop in batch 1495
loop in batch 1500
loop in batch 1505
loop in batch 1510
loop in batch 1515
loop in batch 1520
loop in batch 1525
loop in batch 1530
loop in batch 1535
loop in batch 1540
loop in batch 1545
loop in batch 1550
loop in batch 1555
loop in batch 1560
loop in batch 1565
loop in batch 1570
loop in batch 1575
loop in batch 1580
loop in batch 1585
loop in batch 1590
loop in batch 1595
loop in batch 1600
loop in batch 1605
loop in batch 1610
loop in batch 1615
loop in batch 1620
loop in batch 1625
loop in batch 1630
loop in batch 1635
loop in batch 1640
loop in batch 1645
loop in batch 1650
loop in batch 1655
loop in batch 1660
loop in batch 1665
loop in batch 1670
loop in batc

loop in batch 3575
loop in batch 3580
loop in batch 3585
loop in batch 3590
loop in batch 3595
loop in batch 3600
loop in batch 3605
loop in batch 3610
loop in batch 3615
loop in batch 3620
loop in batch 3625
loop in batch 3630
loop in batch 3635
loop in batch 3640
loop in batch 3645
loop in batch 3650
loop in batch 3655
loop in batch 3660
loop in batch 3665
loop in batch 3670
loop in batch 3675
loop in batch 3680
loop in batch 3685
loop in batch 3690
loop in batch 3695
loop in batch 3700
loop in batch 3705
loop in batch 3710
loop in batch 3715
loop in batch 3720
loop in batch 3725
loop in batch 3730
loop in batch 3735
loop in batch 3740
loop in batch 3745
loop in batch 3750
loop in batch 3755
loop in batch 3760
loop in batch 3765
loop in batch 3770
loop in batch 3775
loop in batch 3780
loop in batch 3785
loop in batch 3790
loop in batch 3795
loop in batch 3800
loop in batch 3805
loop in batch 3810
loop in batch 3815
loop in batch 3820
loop in batch 3825
loop in batch 3830
loop in batc

ReadTimeout: HTTPConnectionPool(host='localhost', port=9000): Read timed out. (read timeout=60)

In [124]:
# load from previous
with open('../result/run2/adjp_sf_batch_60000.json') as json_file:
    previous_dict = json.load(json_file)

In [125]:
len(previous_dict)

16365

In [126]:
# from 65000
process_text_sf_batch_from_prev(df, previous_dict)

number of sentences: 192796
batch 13.0 from 65000
loop in batch 10
loop in batch 20
loop in batch 30
loop in batch 40
loop in batch 50
loop in batch 60
loop in batch 70
loop in batch 80
loop in batch 90
loop in batch 100
loop in batch 110
loop in batch 120
loop in batch 130
loop in batch 140
loop in batch 150
loop in batch 160
loop in batch 170
loop in batch 180
loop in batch 190
loop in batch 200
loop in batch 210
loop in batch 220
loop in batch 230
loop in batch 240
loop in batch 250
loop in batch 260
loop in batch 270
loop in batch 280
loop in batch 290
loop in batch 300
loop in batch 310
loop in batch 320
loop in batch 330
loop in batch 340
loop in batch 350
loop in batch 360
loop in batch 370
loop in batch 380
loop in batch 390
loop in batch 400
loop in batch 410
loop in batch 420
loop in batch 430
loop in batch 440
loop in batch 450
loop in batch 460
loop in batch 470
loop in batch 480
loop in batch 490
loop in batch 500
loop in batch 510
loop in batch 520
loop in batch 530
loop 

loop in batch 4370
loop in batch 4380
loop in batch 4390
loop in batch 4400
loop in batch 4410
loop in batch 4420
loop in batch 4430
loop in batch 4440
loop in batch 4450
loop in batch 4460
loop in batch 4470
loop in batch 4480
loop in batch 4490
loop in batch 4500
loop in batch 4510
loop in batch 4520
loop in batch 4530
loop in batch 4540
loop in batch 4550
loop in batch 4560
loop in batch 4570
loop in batch 4580
loop in batch 4590
loop in batch 4600
loop in batch 4610
loop in batch 4620
loop in batch 4630
loop in batch 4640
loop in batch 4650
loop in batch 4660
loop in batch 4670
loop in batch 4680
loop in batch 4690
loop in batch 4700
loop in batch 4710
loop in batch 4720
loop in batch 4730
loop in batch 4740
loop in batch 4750
loop in batch 4760
loop in batch 4770
loop in batch 4780
loop in batch 4790
loop in batch 4800
loop in batch 4810
loop in batch 4820
loop in batch 4830
loop in batch 4840
loop in batch 4850
loop in batch 4860
loop in batch 4870
loop in batch 4880
loop in batc

loop in batch 3720
loop in batch 3730
loop in batch 3740
loop in batch 3750
loop in batch 3760
loop in batch 3770
loop in batch 3780
loop in batch 3790
loop in batch 3800
loop in batch 3810
loop in batch 3820
loop in batch 3830
loop in batch 3840
loop in batch 3850
loop in batch 3860
loop in batch 3870
loop in batch 3880
loop in batch 3890
loop in batch 3900
loop in batch 3910
loop in batch 3920
loop in batch 3930
loop in batch 3940
loop in batch 3950
loop in batch 3960
loop in batch 3970
loop in batch 3980
loop in batch 3990
loop in batch 4000
loop in batch 4010
loop in batch 4020
loop in batch 4030
loop in batch 4040
loop in batch 4050
loop in batch 4060
loop in batch 4070
loop in batch 4080
loop in batch 4090
loop in batch 4100
loop in batch 4110
loop in batch 4120
loop in batch 4130
loop in batch 4140
loop in batch 4150
loop in batch 4160
loop in batch 4170
loop in batch 4180
loop in batch 4190
loop in batch 4200
loop in batch 4210
loop in batch 4220
loop in batch 4230
loop in batc

loop in batch 3060
loop in batch 3070
loop in batch 3080
loop in batch 3090
loop in batch 3100
loop in batch 3110
loop in batch 3120
loop in batch 3130
loop in batch 3140
loop in batch 3150
loop in batch 3160
loop in batch 3170
loop in batch 3180
loop in batch 3190
loop in batch 3200
loop in batch 3210
loop in batch 3220
loop in batch 3230
loop in batch 3240
loop in batch 3250
loop in batch 3260
loop in batch 3270
loop in batch 3280
loop in batch 3290
loop in batch 3300
loop in batch 3310
loop in batch 3320
loop in batch 3330
loop in batch 3340
loop in batch 3350
loop in batch 3360
loop in batch 3370
loop in batch 3380
loop in batch 3390
loop in batch 3400
loop in batch 3410
loop in batch 3420
loop in batch 3430
loop in batch 3440
loop in batch 3450
loop in batch 3460
loop in batch 3470
loop in batch 3480
loop in batch 3490
loop in batch 3500
loop in batch 3510
loop in batch 3520
loop in batch 3530
loop in batch 3540
loop in batch 3550
loop in batch 3560
loop in batch 3570
loop in batc

loop in batch 2400
loop in batch 2410
loop in batch 2420
loop in batch 2430
loop in batch 2440
loop in batch 2450
loop in batch 2460
loop in batch 2470
loop in batch 2480
loop in batch 2490
loop in batch 2500
loop in batch 2510
loop in batch 2520
loop in batch 2530
loop in batch 2540
loop in batch 2550
loop in batch 2560
loop in batch 2570
loop in batch 2580
loop in batch 2590
loop in batch 2600
loop in batch 2610
loop in batch 2620
loop in batch 2630
loop in batch 2640
loop in batch 2650
loop in batch 2660
loop in batch 2670
loop in batch 2680
loop in batch 2690
loop in batch 2700
loop in batch 2710
loop in batch 2720
loop in batch 2730
loop in batch 2740
loop in batch 2750
loop in batch 2760
loop in batch 2770
loop in batch 2780
loop in batch 2790
loop in batch 2800
loop in batch 2810
loop in batch 2820
loop in batch 2830
loop in batch 2840
loop in batch 2850
loop in batch 2860
loop in batch 2870
loop in batch 2880
loop in batch 2890
loop in batch 2900
loop in batch 2910
loop in batc

...

loop in batch 1490
loop in batch 1500
loop in batch 1510
loop in batch 1520
loop in batch 1530
loop in batch 1540
loop in batch 1550
loop in batch 1560
loop in batch 1570
loop in batch 1580
loop in batch 1590
loop in batch 1600
loop in batch 1610
loop in batch 1620
loop in batch 1630
loop in batch 1640
loop in batch 1650
loop in batch 1660
loop in batch 1670
loop in batch 1680
loop in batch 1690
loop in batch 1700
loop in batch 1710
loop in batch 1720
loop in batch 1730
loop in batch 1740
loop in batch 1750
loop in batch 1760
loop in batch 1770
loop in batch 1780
loop in batch 1790
loop in batch 1800
loop in batch 1810
loop in batch 1820
loop in batch 1830
loop in batch 1840
loop in batch 1850
loop in batch 1860
loop in batch 1870
loop in batch 1880
loop in batch 1890
loop in batch 1900
loop in batch 1910
loop in batch 1920
loop in batch 1930
loop in batch 1940
loop in batch 1950
loop in batch 1960
loop in batch 1970
loop in batch 1980
loop in batch 1990
loop in batch 2000
loop in batc

loop in batch 820
loop in batch 830
loop in batch 840
loop in batch 850
loop in batch 860
loop in batch 870
loop in batch 880
loop in batch 890
loop in batch 900
loop in batch 910
loop in batch 920
loop in batch 930
loop in batch 940
loop in batch 950
loop in batch 960
loop in batch 970
loop in batch 980
loop in batch 990
loop in batch 1000
loop in batch 1010
loop in batch 1020
loop in batch 1030
loop in batch 1040
loop in batch 1050
loop in batch 1060
loop in batch 1070
loop in batch 1080
loop in batch 1090
loop in batch 1100
loop in batch 1110
loop in batch 1120
loop in batch 1130
loop in batch 1140
loop in batch 1150
loop in batch 1160
loop in batch 1170
loop in batch 1180
loop in batch 1190
loop in batch 1200
loop in batch 1210
loop in batch 1220
loop in batch 1230
loop in batch 1240
loop in batch 1250
loop in batch 1260
loop in batch 1270
loop in batch 1280
loop in batch 1290
loop in batch 1300
loop in batch 1310
loop in batch 1320
loop in batch 1330
loop in batch 1340
loop in bat

loop in batch 120
loop in batch 130
loop in batch 140
loop in batch 150
loop in batch 160
loop in batch 170
loop in batch 180
loop in batch 190
loop in batch 200
loop in batch 210
loop in batch 220
loop in batch 230
loop in batch 240
loop in batch 250
loop in batch 260
loop in batch 270
loop in batch 280
loop in batch 290
loop in batch 300
loop in batch 310
loop in batch 320
loop in batch 330
loop in batch 340
loop in batch 350
loop in batch 360
loop in batch 370
loop in batch 380
loop in batch 390
loop in batch 400
loop in batch 410
loop in batch 420
loop in batch 430
loop in batch 440
loop in batch 450
loop in batch 460
loop in batch 470
loop in batch 480
loop in batch 490
loop in batch 500
loop in batch 510
loop in batch 520
loop in batch 530
loop in batch 540
loop in batch 550
loop in batch 560
loop in batch 570
loop in batch 580
loop in batch 590
loop in batch 600
loop in batch 610
loop in batch 620
loop in batch 630
loop in batch 640
loop in batch 650
loop in batch 660
loop in ba

loop in batch 4480
loop in batch 4490
loop in batch 4500
loop in batch 4510
loop in batch 4520
loop in batch 4530
loop in batch 4540
loop in batch 4550
loop in batch 4560
loop in batch 4570
loop in batch 4580
loop in batch 4590
loop in batch 4600
loop in batch 4610
loop in batch 4620
loop in batch 4630
loop in batch 4640
loop in batch 4650
loop in batch 4660
loop in batch 4670
loop in batch 4680
loop in batch 4690
loop in batch 4700
loop in batch 4710
loop in batch 4720
loop in batch 4730
loop in batch 4740
loop in batch 4750
loop in batch 4760
loop in batch 4770
loop in batch 4780
loop in batch 4790
loop in batch 4800
loop in batch 4810
loop in batch 4820
loop in batch 4830
loop in batch 4840
loop in batch 4850
loop in batch 4860
loop in batch 4870
loop in batch 4880
loop in batch 4890
loop in batch 4900
loop in batch 4910
loop in batch 4920
loop in batch 4930
loop in batch 4940
loop in batch 4950
loop in batch 4960
loop in batch 4970
loop in batch 4980
loop in batch 4990
loop in batc

ReadTimeout: HTTPConnectionPool(host='localhost', port=9000): Read timed out. (read timeout=60)

In [181]:
# load from previous
with open('../result/run2/adjp_sf_batch_65000.json') as json_file:
    previous_dict = json.load(json_file)

In [182]:
len(previous_dict)

16365

In [183]:
# from 70000
process_text_sf_batch_from_prev(df, previous_dict)

number of sentences: 192796
batch 13.0 from 65000
loop in batch 10
is there return value 10
len of result list in stanford: 0
loop in batch 20
is there return value 10
len of result list in stanford: 0
loop in batch 30
is there return value 10
len of result list in stanford: 0
loop in batch 40
is there return value 10
len of result list in stanford: 0
loop in batch 50
is there return value 10
len of result list in stanford: 0
loop in batch 60
is there return value 10
len of result list in stanford: 0
loop in batch 70
is there return value 10
len of result list in stanford: 0
loop in batch 80
is there return value 10
len of result list in stanford: 0
loop in batch 90
is there return value 10
len of result list in stanford: 0
loop in batch 100
is there return value 10
len of result list in stanford: 0
loop in batch 110
is there return value 10
len of result list in stanford: 0
loop in batch 120


KeyboardInterrupt: 

In [30]:
parse_dict

{'great': 4,
 'good': 5,
 'bland': 1,
 'really flavourable': 1,
 'fabulous': 1,
 'very pleased with their meals': 1,
 'closer': 1,
 'even recyclable': 1,
 'not - quite - fresh': 1,
 'just abominable': 1,
 'badly charred': 1,
 'coated in black residue which made the already sub': 1,
 'too late': 1,
 'mouth watering': 1,
 'foodgasmic': 1,
 'nice': 1,
 'unforgettable': 1,
 'perfect': 1,
 'so good!great': 1,
 'delicious': 4,
 'too salty and light': 1,
 'friendly enough': 1,
 'pretty expedient': 1,
 'glad we did': 1,
 'incredibly knowledgeable': 1,
 'friendly': 3,
 'helpful': 1,
 'average': 4,
 'large in portion size': 1,
 'a little too oily': 1,
 'spicy': 2,
 'sweet and sour': 2,
 'least favourite': 1,
 'too difficult': 1,
 'unexpected': 1,
 'many': 1,
 'very good': 1,
 'so inexpensive': 1,
 'great for those': 1,
 'overall': 1,
 'so friendly': 1,
 'fair': 1,
 'fair and affordable': 1,
 'willing to do': 1,
 'completely fake': 1,
 'cool and warm': 1,
 'decent enough': 1,
 'truly mediocre': 1

In [22]:
%%time
parse_dict = process_text_sf(df)

number of sentences: 108953
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000


ReadTimeout: HTTPConnectionPool(host='localhost', port=9000): Read timed out. (read timeout=60)

In [23]:
parse_dict 

{'great': 203,
 'good': 275,
 'bland': 16,
 'really flavourable': 2,
 'even alone .': 2,
 'attentive , nice and considerate': 2,
 'way bigger': 5,
 'sure': 60,
 'friendly': 59,
 'so delicious': 8,
 'perfect': 48,
 'amazing': 103,
 'crunchy': 7,
 'clean': 30,
 'fast and friendly': 4,
 'delicious': 97,
 'terrible': 27,
 'fairly close': 1,
 'very clear': 3,
 'more aggressive': 1,
 'unfortunate .': 1,
 'very good': 79,
 'a little salty': 2,
 'horrible': 24,
 'able to contact them when i called them again': 1,
 'more than capable to answer my questions - about a pump that she has not seen , and about a conversation i had with the tech that she did not hear': 1,
 'able to find them on amazon': 1,
 'really good': 55,
 'sweet': 10,
 "really nice inside it does n't seem like your at the cannery": 1,
 'nice': 57,
 'very attentive': 13,
 'so good': 25,
 'fairly quick': 3,
 'far superior to the reeses': 1,
 'higher': 2,
 'better': 64,
 'not interested': 1,
 'polite': 5,
 'forgettable , but good': 

In [24]:
import json

with open('adjp_sf.txt', 'w') as file:
     file.write(json.dumps(parse_dict)) # use `json.loads` to do the reverse

In [169]:
# output from spacy, then put into standford
sent_list = []
adjp_dict_copy = adjp_dict.copy()
for key, value in adjp_dict_copy.items():
    if len(key.split()) > 5:
#         print(key)
        sent_list.append(key)
        
# loop key to delete from adjp_dict_copy
for key in sent_list:
    del adjp_dict_copy[key]

sent_tuple = tuple(sent_list)

parse_result = parse_using_standford(sent_tuple)

for key in parse_result:
    if key in adjp_dict_copy:
        adjp_dict_copy[key] += 1
    else:
        adjp_dict_copy[key] = 1
        
print(parse_result)
adjp_dict_copy

['a little salty', 'only able to contact them when i called them again', 'more than capable to answer my questions - about a pump that she has not seen', 'able to find them on amazon', 'far superior to the reeses', 'awesome']


{'great': 1,
 'good': 2,
 'bland': 1,
 'meh': 1,
 'really flavourable': 1,
 'attentive, nice and considerate': 1,
 'way bigger': 1,
 'friendly': 1,
 'so delicious': 1,
 'perfect': 2,
 'amazing': 4,
 'crunchy': 1,
 'worth the drive': 1,
 'clean': 1,
 'fast and friendly': 1,
 'delicious': 1,
 'this nice': 1,
 'nicest looking': 1,
 'terrible': 1,
 'very clear': 1,
 'more aggressive': 1,
 'unfortunate.': 1,
 'very good': 1,
 'good overall and again': 1,
 'horrible': 1,
 'cash out': 1,
 'really good': 2,
 'sweet': 1,
 'really nice': 1,
 'soooo nice': 1,
 'very attentive': 1,
 'so good': 1,
 'hard': 1,
 'better': 2,
 'not interested': 1,
 'how polite': 1,
 'forgettable, but good': 1,
 'excellent': 2,
 'absolutely top-notch': 1,
 'incredible and friendly': 1,
 'unusually young': 1,
 'albeit sensitive to light': 1,
 'follow-up': 1,
 'very pleased with my vision': 1,
 'fantastic': 1,
 'smoother and moisturized': 1,
 'so elegant and relaxing': 1,
 'available': 1,
 'tidy': 1,
 'so tidy and well-m

In [34]:
print(find_adjp_in_sentence('so tidy and well-maintained'))

so tidy and well-maintained
label tuple ('ADJP',)
so tidy
label tuple ('ADJP',)
so
label tuple ()
tidy
label tuple ()
and
label tuple ()
well-maintained
label tuple ('ADJP',)
well
label tuple ()
-
label tuple ()
maintained
label tuple ()
Number of adjp found: 3
[so tidy and well-maintained, so tidy, well-maintained]


In [ ]:
# ignore

In [23]:
# expression, index
def get_closing_bracket(s, i):
  
    # If input is invalid.
    if s[i] != '(':
        return -1
  
    # Create a deque to use it as a stack.
    d = deque()
  
    # Traverse through all elements
    # starting from i.
    for k in range(i, len(s)):
  
        # Pop a starting bracket
        # for every closing bracket
        if s[k] == ')':
            d.popleft()
  
        # Push all starting brackets
        elif s[k] == '(':
            d.append(s[i])
  
        # If deque becomes empty
        if not d:
            return k
  
    return -1

In [160]:
# for long sentence, try use pos tagging, cut after adjective. do if sentence len > 5?
def pos_tagging(sentence):
    doc = nlp(sentence)
    for token in doc:
        print(token.text, token.pos_, token.tag_)

In [24]:
def get_adjp(parse_tree):
    adjp_list = []
    while (True):
        adjp_index_start = parse_tree.find('(ADJP')
        if (adjp_index_start != -1):
            adjp_index_end = get_closing_bracket(parse_tree, adjp_index_start)
        else:
            break
        phrase = parse_tree[adjp_index_start:adjp_index_end]
        phrase = phrase.split()
        adjp = [i[:-1] for i in phrase if i.find(")") != -1]
        adjp_list.append(' '.join(adjp).replace(')', ''))
        parse_tree = parse_tree[adjp_index_end+1:]
    return adjp_list



In [161]:
pos_tagging('far superior to the reeses pancakes i had tried at overeasy')

far ADV RB
superior ADJ JJ
to ADP IN
the DET DT
reeses NOUN NNS
pancakes NOUN NNS
i PRON PRP
had AUX VBD
tried VERB VBN
at ADP IN
overeasy PROPN JJ


In [46]:
# to test traverse tree sf

sentences = parser.raw_parse_sents(("more than capable to answer my questions - about a pump that she has not seen", "far superior to the reeses pancakes i had tried at overeasy", "sure my next experience was nothing short of awesome"))
print(sentences)

# GUI
for line in sentences:
    for sentence in line:
        print('--------')
        print(sentence)
        print(type(sentence))
        have_adjp, adjp_list = traverse_tree_sf(sentence)
        print(adjp_list)

<generator object GenericCoreNLPParser.raw_parse_sents at 0x7fd7b3639ac0>
--------
(ROOT
  (FRAG
    (ADJP
      (ADVP (JJR more) (IN than))
      (JJ capable)
      (S
        (VP
          (TO to)
          (VP
            (VB answer)
            (NP (PRP$ my) (NNS questions))
            (, -)
            (PP
              (IN about)
              (NP
                (NP (DT a) (NN pump))
                (SBAR
                  (WHNP (WDT that))
                  (S
                    (NP (PRP she))
                    (VP (VBZ has) (RB not) (VP (VBN seen)))))))))))))
<class 'nltk.tree.Tree'>
['more than capable to answer my questions - about a pump that she has not seen']
--------
(ROOT
  (S
    (NP
      (ADJP
        (RB far)
        (JJ superior)
        (PP (IN to) (NP (DT the) (NNS reeses))))
      (NNS pancakes))
    (NP (PRP i))
    (VP (VBD had) (VP (VBN tried) (PP (IN at) (NP (NN overeasy)))))))
<class 'nltk.tree.Tree'>
['far superior to the reeses']
--------
(ROOT
  (S
 

In [162]:
pos_tagging('sure you understand all the costs for out-of-pocket')

sure INTJ JJ
you PRON PRP
understand VERB VBP
all DET PDT
the DET DT
costs NOUN NNS
for ADP IN
out ADV IN
- PUNCT HYPH
of ADP IN
- PUNCT HYPH
pocket NOUN NN


In [163]:
pos_tagging('so tidy and well-maintained')

so ADV RB
tidy ADJ JJ
and CCONJ CC
well ADV RB
- PUNCT HYPH
maintained VERB VBN


In [164]:
pos_tagging('sure my next experience was nothing short of awesome')

sure INTJ JJ
my PRON PRP$
next ADJ JJ
experience NOUN NN
was AUX VBD
nothing PRON NN
short ADJ RB
of ADP IN
awesome ADJ JJ


In [10]:


text = (
  'sure my next experience was nothing short of awesome')


    

# print(output['sentences'][0]['parse'])# tagged output sentence

In [14]:
%%time
output = nlp.annotate(text, properties={
  'annotators': 'parse',
  'outputFormat': 'json'
})
if type(output) is str or type(output) is unicode:
    output = json.loads(output, strict=False)

print(type(output['sentences'][0]['parse']))
test_tree = output['sentences'][0]['parse']

<class 'str'>
CPU times: user 8.07 ms, sys: 3.02 ms, total: 11.1 ms
Wall time: 73.2 ms


In [199]:
print(type(output['sentences'][0]['parse']))# tagged output sentence

<class 'str'>


In [200]:
from IPython.display import display
display(output['sentences'][0]['parse'])

'(ROOT\n  (S\n    (INTJ (UH sure))\n    (NP (PRP$ my) (JJ next) (NN experience))\n    (VP (VBD was)\n      (NP\n        (NP (NN nothing) (NN short))\n        (PP (IN of)\n          (ADJP (JJ awesome)))))))'

In [194]:
def parse_sc(text):
    output = nlp.annotate(text, properties={
  'annotators': 'parse',
  'outputFormat': 'json'})
    
    if type(output) is str or type(output) is unicode:
        output = json.loads(output, strict=False)


    print(output['sentences'][0]['parse'])# tagged output sentence

In [195]:
parse_sc('so tidy and well-maintained')

(ROOT
  (FRAG
    (ADJP
      (ADJP (RB so) (JJ tidy))
      (CC and)
      (ADJP (RB well) (HYPH -) (VBN maintained)))))


In [196]:

def process_text_sc(df):
    text = df['text'].astype('str')
    adjp_dict = {}
    count = 0
    for t in text:
        if count == 20:
            break
        if count%10 == 0:
            print(count)
        count += 1
        
        doc = nlp(t)
        for sent in doc.sents:
            adjp_list = find_adjp_in_sentence(str(sent))
            
            # put the list into adjp dict
            for i in range(len(adjp_list)):
                key = adjp_list[i].text.lower()  # change everything to lower
                if key in adjp_dict:
                    adjp_dict[key] += 1
                else:
                    adjp_dict[key] = 1
    return adjp_dict

In [197]:
process_text(df)

0


TypeError: 'StanfordCoreNLP' object is not callable

nltk & standford

In [1]:
import os
from nltk.parse import stanford
os.environ['STANFORD_PARSER'] = '/Users/cecilialee/Downloads/stanford-parser-full-2020-11-17'
os.environ['STANFORD_MODELS'] = '/Users/cecilialee/Downloads/stanford-parser-full-2020-11-17'

parser = stanford.StanfordParser(model_path="/Users/cecilialee/Downloads/stanford-parser-full-2020-11-17/englishPCFG.ser.gz")
sentences = parser.raw_parse_sents(("Hello, My name is Melroy.", "What is your name?"))
print(sentences)

# GUI
for line in sentences:
    for sentence in line:
#         print(sentence[1])
        sentence.draw()

<ipython-input-1-3e7b02fbc10d>:6: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  parser = stanford.StanfordParser(model_path="/Users/cecilialee/Downloads/stanford-parser-full-2020-11-17/englishPCFG.ser.gz")


IndexError: list index out of range

In [54]:
sentences = parser.raw_parse_sents(("so tidy and well-maintained", "sure my next experience was nothing short of awesome"))
print(sentences)

# GUI
for line in sentences:
    for sentence in line:
        print(sentence)
        sentence.draw()

(ROOT
  (FRAG
    (ADJP
      (ADJP (RB so) (JJ tidy))
      (CC and)
      (ADJP (RB well) (HYPH -) (VBN maintained)))))
(ROOT
  (S
    (INTJ (UH sure))
    (NP (PRP$ my) (JJ next) (NN experience))
    (VP
      (VBD was)
      (NP
        (NP (NN nothing) (NN short))
        (PP (IN of) (ADJP (JJ awesome)))))))


In [29]:
def find_adjp(tree):
    for subtree in tree.subtrees():
        print(subtree)
        print('-----')
        if subtree.label() == 'ADJP':
            print(subtree.leaves())

In [30]:
sentences = parser.raw_parse_sents(("so tidy and well-maintained", "far superior to the reeses pancakes i had tried at overeasy", "sure my next experience was nothing short of awesome"))
print(sentences)

# GUI
for line in sentences:
    for sentence in line:
        print(sentence)
#         print(type(sentence))
        find_adjp(sentence)
        
        #sentence.draw()

(ROOT
  (FRAG
    (ADJP
      (ADJP (RB so) (JJ tidy))
      (CC and)
      (ADJP (RB well) (HYPH -) (VBN maintained)))))
(ROOT
  (FRAG
    (ADJP
      (ADJP (RB so) (JJ tidy))
      (CC and)
      (ADJP (RB well) (HYPH -) (VBN maintained)))))
-----
(FRAG
  (ADJP
    (ADJP (RB so) (JJ tidy))
    (CC and)
    (ADJP (RB well) (HYPH -) (VBN maintained))))
-----
(ADJP
  (ADJP (RB so) (JJ tidy))
  (CC and)
  (ADJP (RB well) (HYPH -) (VBN maintained)))
-----
['so', 'tidy', 'and', 'well', '-', 'maintained']
(ADJP (RB so) (JJ tidy))
-----
['so', 'tidy']
(RB so)
-----
(JJ tidy)
-----
(CC and)
-----
(ADJP (RB well) (HYPH -) (VBN maintained))
-----
['well', '-', 'maintained']
(RB well)
-----
(HYPH -)
-----
(VBN maintained)
-----
(ROOT
  (S
    (NP
      (ADJP
        (RB far)
        (JJ superior)
        (PP (IN to) (NP (DT the) (NNS reeses))))
      (NNS pancakes))
    (NP (PRP i))
    (VP (VBD had) (VP (VBN tried) (PP (IN at) (NP (NN overeasy)))))))
(ROOT
  (S
    (NP
      (ADJP
        (RB f

In [31]:
sentences = parser.raw_parse_sents(("so tidy and well-maintained", "far superior to the reeses pancakes i had tried at overeasy", "sure my next experience was nothing short of awesome"))
print(sentences)

# GUI
for line in sentences:
    for sentence in line:
        print(sentence)
#         print(type(sentence))
        traverse_tree(sentence)
        
        #sentence.draw()

(ROOT
  (FRAG
    (ADJP
      (ADJP (RB so) (JJ tidy))
      (CC and)
      (ADJP (RB well) (HYPH -) (VBN maintained)))))
tree: (ROOT
  (FRAG
    (ADJP
      (ADJP (RB so) (JJ tidy))
      (CC and)
      (ADJP (RB well) (HYPH -) (VBN maintained)))))
tree: (FRAG
  (ADJP
    (ADJP (RB so) (JJ tidy))
    (CC and)
    (ADJP (RB well) (HYPH -) (VBN maintained))))
tree: (ADJP
  (ADJP (RB so) (JJ tidy))
  (CC and)
  (ADJP (RB well) (HYPH -) (VBN maintained)))
tree: (ADJP (RB so) (JJ tidy))
tree: (RB so)
tree: (JJ tidy)
tree: (CC and)
tree: (ADJP (RB well) (HYPH -) (VBN maintained))
tree: (RB well)
tree: (HYPH -)
tree: (VBN maintained)
(ROOT
  (S
    (NP
      (ADJP
        (RB far)
        (JJ superior)
        (PP (IN to) (NP (DT the) (NNS reeses))))
      (NNS pancakes))
    (NP (PRP i))
    (VP (VBD had) (VP (VBN tried) (PP (IN at) (NP (NN overeasy)))))))
tree: (ROOT
  (S
    (NP
      (ADJP
        (RB far)
        (JJ superior)
        (PP (IN to) (NP (DT the) (NNS reeses))))
      (NNS 

In [ ]:
sentences = parser.raw_parse_sents(("more than capable to answer my questions - about a pump that she has not seen, and about a conversation i had with the tech that she did not hear able to find them on amazon", "far superior to the reeses pancakes i had tried at overeasy", "sure my next experience was nothing short of awesome"))
print(sentences)

# GUI
for line in sentences:
    for sentence in line:
#         print(sentence)
#         print(type(sentence))
#         traverse_tree(sentence)
        
        sentence.draw()
        str_sent = ' '.join(str(sentence).split())
#         print(str_sent)
        print(get_adjp(str_sent))

In [69]:
print(parse_result)

['a little salty', 'only able to contact them when i called them again', 'more than capable to answer my questions - about a pump that she has not seen', 'able to find them on amazon', 'far superior to the reeses', 'awesome']


In [52]:
# try parsing with just nltk
def process_text_sf(df):
    text = df['text'].astype('str')
    adjp_dict = {}
    count = 0
    for t in text:
        if count == 20:
            break
        if count%10 == 0:
            print(count)
        count += 1
        
        sentences = parser.raw_parse_sents((t))
        for line in sentences:
            for sentence in line:
                str_sent = ' '.join(str(sentence).split())
                adjp_list = get_adjp(str_sent)
                print(adjp_list)
            
        # put the list into adjp dict
        for i in range(len(adjp_list)):
            key = adjp_list[i].text.lower()  # change everything to lower
            if key in adjp_dict:
                adjp_dict[key] += 1
            else:
                adjp_dict[key] = 1
    return adjp_dict

In [53]:
dict_test = process_text_sf(df)

0
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [51]:
dict_test

{}

In [24]:
print(type(sentences))

<class 'list_iterator'>


In [12]:
%%time
sentences = parser.raw_parse_sents(("sure my next experience was nothing short of awesome"))

CPU times: user 6.2 ms, sys: 38.2 ms, total: 44.4 ms
Wall time: 1.61 s


In [23]:
# traverse tree
import nltk
from nltk.tree import Tree
# tree = nltk.tree.Tree.fromstring(s)
def traverse_tree(tree):
    print("tree:", tree)
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverse_tree(subtree)

In [22]:
tree = nltk.tree.Tree.fromstring(test_tree)
traverse_tree(tree)

tree: (ROOT
  (S
    (INTJ (UH sure))
    (NP (PRP$ my) (JJ next) (NN experience))
    (VP
      (VBD was)
      (NP
        (NP (NN nothing) (NN short))
        (PP (IN of) (ADJP (JJ awesome)))))))
tree: (S
  (INTJ (UH sure))
  (NP (PRP$ my) (JJ next) (NN experience))
  (VP
    (VBD was)
    (NP
      (NP (NN nothing) (NN short))
      (PP (IN of) (ADJP (JJ awesome))))))
tree: (INTJ (UH sure))
tree: (UH sure)
tree: (NP (PRP$ my) (JJ next) (NN experience))
tree: (PRP$ my)
tree: (JJ next)
tree: (NN experience)
tree: (VP
  (VBD was)
  (NP (NP (NN nothing) (NN short)) (PP (IN of) (ADJP (JJ awesome)))))
tree: (VBD was)
tree: (NP (NP (NN nothing) (NN short)) (PP (IN of) (ADJP (JJ awesome))))
tree: (NP (NN nothing) (NN short))
tree: (NN nothing)
tree: (NN short)
tree: (PP (IN of) (ADJP (JJ awesome)))
tree: (IN of)
tree: (ADJP (JJ awesome))
tree: (JJ awesome)


In [ ]:
def traverse(t):
    np_trees = []
    try:
        t.label()
    except AttributeError:
        return

    if t.label() == "ADJP":
        current = t
        while current.parent() is not None:

            while current.left_sibling() is not None:

                if current.left_sibling().label() == "NP":
                    np_trees.append(current.left_sibling())

                current = current.left_sibling()

            current = current.parent()

    for child in t:
        traverse(child)

# ignore

In [47]:
list(sent._.parse_string)[4]

'N'

In [75]:
# make sure it is while loop to make sure get all the adjp in the sentence
adjp = get_adjp(parse)

In [76]:
print(adjp)

['most powerful']


In [64]:
i = adjp.split()

In [65]:
print(i)

['(ADJP', '(RBS', 'most)', '(JJ', 'powerful)']


In [78]:
sentence = 'Frustrated, overworked and underpaid employees picketed for hours. The exhausted and overworked man took a well-deserved break. Young, otherwise healthy people are most likely to recover quickly.'
sentence_doc = nlp(sentence)


/Users/cecilialee/miniconda3/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [79]:
sent1 = list(sentence_doc.sents)[0]
print(sent1._.parse_string)
adjp1 = get_adjp(sent1._.parse_string)
print(adjp1)

(S (NP (ADJP (JJ Frustrated) (, ,) (JJ overworked) (CC and) (JJ underpaid)) (NNS employees)) (VP (VBD picketed) (PP (IN for) (NP (NNS hours)))) (. .))
['Frustrated , overworked and underpaid']


In [33]:
def get_adjp_from_sentence(sentence):
    sentence_doc = nlp(sentence)
    sent = list(sentence_doc.sents)[0]
#     print(sent._.parse_string)
    adjp = get_adjp(sent._.parse_string)
#     print(adjp)
    return adjp

In [109]:
get_adjp_from_sentence('The exhausted and overworked man took a well-deserved break.')

['exhausted and overworked', 'well - deserved']

In [86]:
get_adjp_from_sentence('Young, otherwise healthy people are most likely to recover quickly.')

(S (NP (JJ Young) (, ,) (ADJP (RB otherwise) (JJ healthy)) (NNS people)) (VP (VBP are) (ADJP (RBS most) (JJ likely) (S (VP (TO to) (VP (VB recover) (ADVP (RB quickly))))))) (. .))
['otherwise healthy', 'most likely to recover quickly']


In [85]:
get_adjp_from_sentence('The new outfit was very pricey but really beautiful.')

(S (NP (DT The) (JJ new) (NN outfit)) (VP (VBD was) (ADJP (ADJP (RB very) (JJ pricey)) (CC but) (ADJP (RB really) (JJ beautiful)))) (. .))
['very pricey but really beautiful']


In [103]:
get_adjp_from_sentence('more than capable to answer my questions - about a pump that SHE HAS NOT SEEN , and about a conversation I had with the tech THAT SHE DID NOT HEAR')

(ADJP (ADVP (RBR more) (IN than)) (JJ capable) (S (VP (TO to) (VP (VB answer) (NP (NP (PRP$ my) (NNS questions)) (: -) (PP (PP (IN about) (NP (NP (DT a) (NN pump)) (SBAR (WHNP (WDT that)) (S (NP (PRP SHE)) (VP (VBZ HAS) (RB NOT) (VP (VBN SEEN))))))) (, ,) (CC and) (PP (IN about) (NP (NP (DT a) (NN conversation)) (SBAR (S (NP (PRP I)) (VP (VBD had) (PP (IN with) (NP (DT the) (NN tech))) (SBAR (WHNP (WDT THAT)) (S (NP (PRP SHE)) (VP (VBD DID) (RB NOT) (VP (VB HEAR))))))))))))))))
['more than capable to answer my questions - about a pump that SHE HAS NOT SEEN , and about a conversation I had with the tech THAT SHE DID NOT HEAR']


['more than capable to answer my questions - about a pump that SHE HAS NOT SEEN , and about a conversation I had with the tech THAT SHE DID NOT HEAR']

In [105]:
get_adjp_from_sentence('Far superior to the Reeses pancakes I had tried at OverEasy')

(ADJP (RB Far) (JJ superior) (PP (IN to) (NP (NP (DT the) (NNP Reeses) (NNS pancakes)) (SBAR (S (NP (PRP I)) (VP (VBD had) (VP (VBN tried) (PP (IN at) (NP (NNP OverEasy))))))))))
['Far superior to the Reeses pancakes I had tried at OverEasy']


['Far superior to the Reeses pancakes I had tried at OverEasy']

In [111]:
%%time
adjp_dict = {}
all_adjp_list = []
count = 0
for t in text:
    if count%10 == 0:
        print(count)
    count += 1
    doc = nlp(t)
    for sent in doc.sents:
        ret = get_adjp_from_sentence(str(sent))
        all_adjp_list.extend(ret)
        
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [133]:
len(all_adjp_list)

66435

In [ ]:
adjp_dict = {}
all_adjp_list = []
count = 0
for t in text:
    if count%10 == 0:
        print(count)
    count += 1
    doc = nlp(t)
    for sent in doc.sents:
        ret = get_adjp_from_sentence(str(sent))
        all_adjp_list.extend(ret)

In [102]:
print(all_adjp_list)

['Great', 'good', 'bland', 'meh', 'really flavourable', 'attentive , nice and considerate', 'way bigger', 'sure if the guy was the owner', 'friendly', 'so delicious', 'perfect', 'amazing', 'crunchy', 'worth the drive', 'clean', 'fast and friendly', 'delicious', 'this nice', 'nicest looking', 'terrible', 'VERY clear', 'more aggressive', 'Unfortunate .', 'Very good', 'A little salty on some items during our first visit', 'good overall and again', 'horrible', 'only able to contact them when I called them AGAIN', 'more than capable to answer my questions - about a pump that SHE HAS NOT SEEN , and about a conversation I had with the tech THAT SHE DID NOT HEAR', 'cash out', 'able to find them on Amazon', 'really good', 'sweet', 'really nice', 'good', 'really good', 'soooo nice', 'very attentive', 'so good', 'amazing', 'perfect', 'hard', 'Far superior to the Reeses pancakes I had tried at OverEasy']


In [96]:
#select all the text
text = df['text'].astype('str')
count = 0
for s in text:
    if count > 10:
        break
    count += 1
    print(s)
#join all the text into one string

print('------------')
doc = nlp(text[0])
count = 0
for sent in doc.sents:
    if count > 10:
        break
    print('.../')
    print(sent)
    count += 1

# print('Number of sentences {}'.format(len(sentences)))

We had my Mother's Birthday Party here on 10/29/16. What a Great time we all had. The food, music and waiters were Great!!! Thanks Lyles!!!
Good Korean grill near Eaton Centre. The marinate is good. We got beef, ox liver, salmon, fish fillet, chicken, pork, pork belly. The fish fillet was bland and liver was meh. Salmon and chicken was really flavourable. Such a fun place to eat at for a date or group of friends. Even alone. No judgments here. 
The staff is attentive, nice and considerate. Bigger groups will most likely be seated on the second floor which is way bigger.
Caution: will smell like BBQ grill after.
Was recommended to try this place by few people and today was my first time here. All I can say is, I am coming back very soon.

SERVICE
Wasn't sure if the guy was the owner but he was friendly and talked story while we waited for our food. Loved it!! Food came out within 10 min. 

FOOD
Tried hamburger steak and it was so delicious. Gravy/sauce they put on the hamburger steak wa